# run script for experiment
This is the batch script for EG prediction used for the experiments.
themanual page iof EG_prediction.py is the follwing:

In [1]:
!python EG_prediction.py -h

usage: EG_prediction.py [-h] -i <inputfile> [<inputfile> ...]
                        [-X <excludelabels> [<excludelabels> ...]]
                        [-L <labelname>] -l <labelfile> [-A <aliases>]
                        [-b <seed>] [-r <repeat>] [-f <folds>] [-j <jobs>]
                        [-B] [-sf <subfolds>] [-P] [-ba] [-fx]
                        [-n <normalize>] [-o <outfile>] [-s <scorefile>]

PLOS COMPBIO

options:
  -h, --help            show this help message and exit
  -i <inputfile> [<inputfile> ...], --inputfile <inputfile> [<inputfile> ...]
                        input attribute filename list
  -X <excludelabels> [<excludelabels> ...], --excludelabels <excludelabels> [<excludelabels> ...]
                        labels to exclude (default NaN, values any list)
  -L <labelname>, --labelname <labelname>
                        label name (default label)
  -l <labelfile>, --labelfile <labelfile>
                        label filename
  -A <aliases>, --aliases <alias

In [4]:
datapath = "../data"
tissue = "Lung"                               # or 'Lung'
labelfile = f"{tissue}_HELP.csv"                # label filename
aliases = "-A \"{'aE': 'NE', 'sNE':'NE'}\""    # dictionary for renaming labels before prediction: es. {'oldlabel': 'newlabel'}
#aliases = ""
#excludeflags = "-X aE"                          # label to remove: es. -X aE (for E vs sNE problem)
excludeflags = ""                          # label to remove: es. -X aE (for E vs sNE problem)
njobs = "-1"                                    # parallelism level: -1 = all cpus, 1 = sequential
sfolds = "4"                                    # dataset subsampling factor: es: 4 for 1:4 ratio of <minority-class>:<majority-class>
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv  {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -n std -ba -sf {sfolds} {excludeflags}  -j {njobs} -P

- replacing label aE with NE
- replacing label sNE with NE
- removing label []
Subsampling with factor 1:4...
Majority NE 16696 minoriy E 1235
Subsampling with factor 1:4
[Lung_BIO.csv] found 0 Nan...
[Lung_BIO.csv] Normalization with std ...
[Lung_CCcfs.csv] found 0 Nan...
[Lung_CCcfs.csv] Normalization with std ...
[Lung_EmbN2V_128.csv] found 0 Nan...
[Lung_EmbN2V_128.csv] Normalization with std ...
5607 labeled genes over a total of 6175
(5607, 3454) data input
Running par on 8 cpus...
{'E': 0, 'NE': 1}
label
NE       4449
E        1158
Name: count, dtype: int64
{'E': 0, 'NE': 1}
label
NE       4449
E        1158
Name: count, dtype: int64
{'E': 0, 'NE': 1}
label
NE       4449
E        1158
Name: count, dtype: int64
Classification with LightGBM...
{'E': 0, 'NE': 1}
label
NE       4449
E        1158
Name: count, dtype: int64
{'E': 0, 'NE': 1}
label
NE       4449
E        1158
Name: count, dtype: int64
Classification with LightGBM...
{'E': 0, 'NE': 1}
label
NE       4449
E        1158


In [17]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 1 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Lung_BIO.csv Lung_CCcfs.csv Lung_EmbN2V_128.csv
LABEL: Lung_HELP.csv DISTRIB: E : 1158, NE: 1123
SUBSAMPLE: 1:1
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9517±0.0096                 |
| Accuracy    | 0.8879±0.0141                 |
| BA          | 0.8876±0.0141                 |
| Sensitivity | 0.9043±0.0179                 |
| Specificity | 0.8709±0.0214                 |
| MCC         | 0.7762±0.0281                 |
| CM          | [[10472, 1108], [1450, 9780]] |
+-------------+-------------------------------+


In [18]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 2 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Lung_BIO.csv Lung_CCcfs.csv Lung_EmbN2V_128.csv
LABEL: Lung_HELP.csv DISTRIB: E : 1158, NE: 2236
SUBSAMPLE: 1:2
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9533±0.0074                 |
| Accuracy    | 0.8882±0.0104                 |
| BA          | 0.8772±0.0116                 |
| Sensitivity | 0.8425±0.0222                 |
| Specificity | 0.9119±0.0138                 |
| MCC         | 0.7525±0.0227                 |
| CM          | [[9756, 1824], [1970, 20390]] |
+-------------+-------------------------------+


In [19]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 4 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Lung_BIO.csv Lung_CCcfs.csv Lung_EmbN2V_128.csv
LABEL: Lung_HELP.csv DISTRIB: E : 1158, NE: 4449
SUBSAMPLE: 1:4
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9565±0.0061                 |
| Accuracy    | 0.9126±0.0081                 |
| BA          | 0.8684±0.0145                 |
| Sensitivity | 0.7932±0.0286                 |
| Specificity | 0.9437±0.0077                 |
| MCC         | 0.7345±0.0248                 |
| CM          | [[9185, 2395], [2506, 41984]] |
+-------------+-------------------------------+


In [20]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 6 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Lung_BIO.csv Lung_CCcfs.csv Lung_EmbN2V_128.csv
LABEL: Lung_HELP.csv DISTRIB: E : 1158, NE: 6704
SUBSAMPLE: 1:6
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9571±0.0059                 |
| Accuracy    | 0.9268±0.0051                 |
| BA          | 0.8594±0.0125                 |
| Sensitivity | 0.7639±0.0258                 |
| Specificity | 0.9549±0.0053                 |
| MCC         | 0.7118±0.0199                 |
| CM          | [[8846, 2734], [3021, 64019]] |
+-------------+-------------------------------+


In [21]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 8 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Lung_BIO.csv Lung_CCcfs.csv Lung_EmbN2V_128.csv
LABEL: Lung_HELP.csv DISTRIB: E : 1158, NE: 8913
SUBSAMPLE: 1:8
+-------------+-------------------------------+
|             | measure                       |
|-------------+-------------------------------|
| ROC-AUC     | 0.9569±0.0060                 |
| Accuracy    | 0.9334±0.0044                 |
| BA          | 0.8478±0.0130                 |
| Sensitivity | 0.7367±0.0265                 |
| Specificity | 0.9589±0.0044                 |
| MCC         | 0.6805±0.0208                 |
| CM          | [[8531, 3049], [3659, 85471]] |
+-------------+-------------------------------+


In [22]:
!python EG_prediction.py -i {datapath}/{tissue}_BIO.csv {datapath}/{tissue}_CCcfs.csv {datapath}/{tissue}_EmbN2V_128.csv -l {datapath}/{labelfile} {aliases} -ba -sf 10 {excludeflags}  -j {njobs} -P -B

METHOD: LGBM	MODE: prob	BALANCE: yes
PROBL: E vs NE
INPUT: Lung_BIO.csv Lung_CCcfs.csv Lung_EmbN2V_128.csv
LABEL: Lung_HELP.csv DISTRIB: E : 1158, NE: 11143
SUBSAMPLE: 1:10
+-------------+--------------------------------+
|             | measure                        |
|-------------+--------------------------------|
| ROC-AUC     | 0.9565±0.0066                  |
| Accuracy    | 0.9402±0.0040                  |
| BA          | 0.8413±0.0142                  |
| Sensitivity | 0.7194±0.0287                  |
| Specificity | 0.9631±0.0036                  |
| MCC         | 0.6613±0.0225                  |
| CM          | [[8330, 3250], [4107, 107323]] |
+-------------+--------------------------------+
